In [2]:
def extract_first_char(s: str):
    if s is None:
        raise ValueError("argument cannot be None")
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if len(s) == 0:
        raise ValueError("argument cannot be an empty string")
    return s[0]

In [3]:
try:
    extract_first_char(100)
except TypeError as ex:
    print(ex)

argument must be a string


In [4]:
def extract_first_char(s: str):
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if not s:
        raise ValueError("argument cannot be an empty string")
    return s[0]

In [5]:
try:
    extract_first_char(100)
except TypeError as ex:
    print(ex)

argument must be a string


In [6]:
try:
    extract_first_char("")
except ValueError as ex:
    print(ex)

argument cannot be an empty string


In [7]:
try:
    extract_first_char(None)
except TypeError as ex:
    print(ex)

argument must be a string


In [8]:
from typing import Annotated
from pydantic import Field, ValidationError

In [9]:
NonEmptyString = Annotated[str, Field(min_length=1)]

In [10]:
def extract_first_char(s: NonEmptyString):
    if s is None:
        raise ValueError("argument cannot be none")
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if len(s) == 0:
        raise ValueError("argument cannot be an empty string")
    return[0]

In [11]:
# Pydantic decorator to run the validation automatically 
from pydantic import validate_call

@validate_call
def extract_first_char(s: NonEmptyString):
    return s[0]

In [12]:
extract_first_char("abc")

'a'

In [13]:
try:
    extract_first_char(None)
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [14]:
try:
    extract_first_char("")
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short


In [15]:
try:
    extract_first_char(100)
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [16]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse

def make_utc(dt: datetime) -> datetime:
    print("make_utc called...")
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    print("parse_datetime called...")
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

In [17]:
from pydantic import BeforeValidator, AfterValidator

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [20]:
@validate_call
def func(dt: DateTimeUTC):
    return dt.isoformat()

In [21]:
func("2020/1/1 3pm")

parse_datetime called...
make_utc called...


'2020-01-01T15:00:00+00:00'

In [22]:
def func(dt: datetime | str):
    try:
        dt = parse(dt)
    except Exception as ex:
        raise ValueError(str(ex))
    dt = make_utc(dt)
    return dt

In [24]:
func("2020/1/1 3pm")

make_utc called...


datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)